<a href="https://colab.research.google.com/github/ranaurek/Quantum-Graph-Coloring/blob/main/diagrams.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit import Aer, execute
from qiskit.circuit import Parameter
import numpy as np

In [ ]:
node1 = QuantumRegister(2, name='node_1')
node2 = QuantumRegister(2, name='node_2')
anc1 = QuantumRegister(1, name='anc_1')
anc2 = QuantumRegister(1, name='anc_2')
clause_qubit = QuantumRegister(1, name='clause')

qc = QuantumCircuit(node1,node2,anc1,anc2,clause_qubit)


qc.cx(node1[0], anc1)
qc.cx(node2[0], anc1) # turn on anc1 if a and c are different

qc.cx(node1[1], anc2)
qc.cx(node2[1], anc2) 

qc.cx(anc1,clause_qubit)   # turn on output if anc1 is on 
qc.cx(anc2,clause_qubit)   # turn on output if anc2 is on

qc.ccx(anc1,anc2,clause_qubit)    # turn on output if anc1 and anc2 is on (to correct possible mistake if both anc1 and anc2 were on originally)

qc.draw()

node_1_0: ──■────────────────────────────────
            │                                
node_1_1: ──┼────■───────────────────────────
            │    │                           
node_2_0: ──┼────┼────■──────────────────────
            │    │    │                      
node_2_1: ──┼────┼────┼────■─────────────────
          ┌─┴─┐  │  ┌─┴─┐  │                 
   anc_1: ┤ X ├──┼──┤ X ├──┼────■─────────■──
          └───┘┌─┴─┐└───┘┌─┴─┐  │         │  
   anc_2: ─────┤ X ├─────┤ X ├──┼────■────■──
               └───┘     └───┘┌─┴─┐┌─┴─┐┌─┴─┐
  clause: ────────────────────┤ X ├┤ X ├┤ X ├
                              └───┘└───┘└───┘

In [ ]:
lb = 3 (# log b gates) 2^3 = 8 bits –> 256 colors
regs = [] 
node1 = QuantumRegister(2**lb, name='node_1 p')
node2 = QuantumRegister(2**lb, name='node_2 p')

regs+=[node1]
regs+=[node2]

ancs = []
for lvl in range(0,lb+1):
    if(lvl==lb):
        ancs.append(QuantumRegister(2**(lb-lvl), name="output"))
    else:
        ancs.append(QuantumRegister(2**(lb-lvl), name='level '+str(lvl)+" anc"))
    regs+=[ancs[lvl]]
    
qc = QuantumCircuit(*regs)

for i in range(0,2**lb):
    qc.cx(node1[i],ancs[0][i])
    qc.cx(node2[i],ancs[0][i]) 

for lvl in range(0,lb):
    print(len(ancs[lvl]))
    for i in range(0,len(ancs[lvl]),2):
        qc.cx(ancs[lvl][i],ancs[lvl+1][i//2])
        qc.cx(ancs[lvl][i+1],ancs[lvl+1][i//2])
        qc.ccx(ancs[lvl][i],ancs[lvl][i+1],ancs[lvl+1][i//2])
    
    

qc.draw(fold=500)
    
# given an edge between nodes, we need 2 * b cx gates for the first level (into the ancilla layers)
# b cx in the next
# b/2 cx in the next... 

8
4
2


node_1 p_0: ──■──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                 │                                                                                                                                                                                      
   node_1 p_1: ──┼────■─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                 │    │                                                                                                                                                                                 
   node_1 p_2: ──┼────┼────■────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                 │    │    │                                                                                                                                                                            
   node_1 p_3: ──┼────┼────┼────■───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                 │    │    │    │                                                                                                                                                                       
   node_1 p_4: ──┼────┼────┼────┼────■──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                 │    │    │    │    │                                                                                                                                                                  
   node_1 p_5: ──┼────┼────┼────┼────┼────■─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                 │    │    │    │    │    │                                                                                                                                                             
   node_1 p_6: ──┼────┼────┼────┼────┼────┼────■────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                 │    │    │    │    │    │    │                                                                                                                                                        
   node_1 p_7: ──┼────┼────┼────┼────┼────┼────┼────■───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                 │    │    │    │    │    │    │    │                                                                                                                                                   
   node_2 p_0: ──┼────┼────┼────┼────┼────┼────┼────┼────■──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                 │    │    │    │    │    │    │    │    │                                                                                                                                              
   node_2 p_1: ──┼────┼────┼────┼────┼────┼────┼────┼────┼────■─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                 │    │    │    │    │    │    │    │    │    │                                                                                                                         